In [4]:
#importando bibliotecas
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver import ChromeOptions
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from time import sleep
import pandas as pd
from datetime import date

In [7]:
class Jogos:

    def __init__(self, ano):
        self.ano = ano
        opts = ChromeOptions()
        servico=Service(ChromeDriverManager().install())
        driver=webdriver.Chrome(service=servico, options=opts)
        self.driver = driver
    
    def _rodada_atual_func(self):
        """definir função para aquisitar a rodada atual"""
        try:
            self.rodada_atual = int(self.driver.find_element(By.XPATH,'/html/body/div[2]/main/div[2]/div/section[1]/section/nav/span[2]').text[:2])
        except:
            self.rodada_atual = int(self.driver.find_element(By.XPATH,'/html/body/div[2]/main/div[2]/div/section[1]/section/nav/span[2]').text[:1])

    def _primeira_rodada(self):
        """retornar para a primeira rodada"""
        while True:
            check1 = 0
            try:
                while True:
                    self.driver.find_element(By.XPATH,"/html/body/div[2]/main/div[2]/div/section[1]/section/nav/span[1]").click()
                    Jogos._rodada_atual_func(self)
                    if self.rodada_atual == 1:
                        break
                break
            except:
                if check1 == 10:
                    break
                else:
                    sleep(1)
                    check1 += 1

    def tabela_partidas(self):
        """criar tabela de jogos com datas, times mandantes e times visitantes"""
        
        self.driver.get("https://ge.globo.com/futebol/brasileirao-serie-a/")
        self.driver.maximize_window()
        self.driver.execute_script("window.scrollTo(0, 300);")
        sleep(3)
        Jogos._primeira_rodada(self)
        jogos_df = pd.DataFrame({'Datas':[], 'Time Mandante': [], 'Time Visitante': [], 'Rodada': []})
        while True:
            check1 = 0
            try:
                while True:
                    datas_lista, mandante_lista, visitante_lista, rodada = [], [], [], []
                    for partida in range(10):
                        data = self.driver.find_element(By.XPATH,f'/html/body/div[2]/main/div[2]/div/section[1]/section/ul/li[{partida+1}]/div/a/div/div[1]/span[2]').text
                        datas_lista.append(date(self.ano, int(data.split('/')[-1]), int(data.split('/')[0])))
                        mandante_lista.append(self.driver.find_element(By.XPATH,f"/html/body/div[2]/main/div[2]/div/section[1]/section/ul/li[{partida+1}]/div/a/div/div[2]/div[1]/span[1]").get_attribute('title'))
                        visitante_lista.append(self.driver.find_element(By.XPATH,f"/html/body/div[2]/main/div[2]/div/section[1]/section/ul/li[{partida+1}]/div/a/div/div[2]/div[3]/span[1]").get_attribute('title'))
                        rodada.append(int(self.rodada_atual))

                    df = pd.DataFrame({'Datas':datas_lista, 'Time Mandante': mandante_lista, 'Time Visitante': visitante_lista, 'Rodada': rodada})
                    jogos_df = pd.concat([jogos_df, df])
                    Jogos._rodada_atual_func(self)
                    if self.rodada_atual == 38:
                        break
                    else:
                        self.driver.find_element(By.XPATH,"/html/body/div[2]/main/div[2]/div/section[1]/section/nav/span[3]").click()
                break
            except:
                if check1 == 15:
                    break
                else:
                    sleep(1)
                    check1+=1
                   
        self.tabela_partidas = jogos_df
        self.driver.quit()

def main(ano):

    jogos_objeto = Jogos(ano)
    jogos_objeto.tabela_partidas()
    jogos_df = jogos_objeto.tabela_partidas

    return jogos_df



In [8]:
jogos_df = main(2023)

jogos_df

,Datas,Time Mandante,Time Visitante,Rodada
0,2023-04-15,Palmeiras,Cuiabá,1.0
1,2023-04-15,América-MG,Fluminense,1.0
2,2023-04-15,Botafogo,São Paulo,1.0
3,2023-04-15,Bragantino,Bahia,1.0
4,2023-04-15,Athletico-PR,Goiás,1.0
...,...,...,...,...
5,2023-12-06,Cruzeiro,Palmeiras,37.0
6,2023-12-06,Internacional,Botafogo,37.0
7,2023-12-06,Coritiba,Corinthians,37.0
8,2023-12-06,Bahia,Atlético-MG,37.0


In [10]:
data_teste = date(2023, 6, 6)

In [11]:
data_teste

datetime.date(2023, 6, 6)

In [ ]:
#entrar na página do brasileiro série A
opts = ChromeOptions()
servico=Service(ChromeDriverManager().install())
driver=webdriver.Chrome(service=servico, options=opts)

driver.get("https://ge.globo.com/futebol/brasileirao-serie-a/")

#input do ano 
ano = 2023

#definir função para aquisitar a rodada atual
def rodada_atual_func():
    try:
        rodada_atual = int(driver.find_element(By.XPATH,'/html/body/div[2]/main/div[2]/div/section[1]/section/nav/span[2]').text[:2])
    except:
        rodada_atual = int(driver.find_element(By.XPATH,'/html/body/div[2]/main/div[2]/div/section[1]/section/nav/span[2]').text[:1])
    return rodada_atual

#retornar para a primeira rodada
while True:
    driver.find_element(By.XPATH,"/html/body/div[2]/main/div[2]/div/section[1]/section/nav/span[1]").click()
    sleep(1)
    rd = rodada_atual_func()
    if rd == 1:
        break

#criar tabela de jogos com datas, times mandantes e times visitantes
jogos_df = pd.DataFrame({'Datas':[], 'Time Mandante': [], 'Time Visitante': []})
while True:
    datas_lista, mandante_lista, visitante_lista = [], [], []
    for partida in range(10):
        data = driver.find_element(By.XPATH,f'/html/body/div[2]/main/div[2]/div/section[1]/section/ul/li[{partida+1}]/div/a/div/div[1]/span[2]').text
        datas_lista.append(date(ano, int(data.split('/')[-1]), int(data.split('/')[0])))
        mandante_lista.append(driver.find_element(By.XPATH,f"/html/body/div[2]/main/div[2]/div/section[1]/section/ul/li[{partida+1}]/div/a/div/div[2]/div[1]/span[1]").get_attribute('title'))
        visitante_lista.append(driver.find_element(By.XPATH,f"/html/body/div[2]/main/div[2]/div/section[1]/section/ul/li[{partida+1}]/div/a/div/div[2]/div[3]/span[1]").get_attribute('title'))

    df = pd.DataFrame({'Datas':datas_lista, 'Time Mandante': mandante_lista, 'Time Visitante': visitante_lista})
    jogos_df = pd.concat([jogos_df, df])

    rd = rodada_atual_func()
    if rd == 38:
        break
    else:
        driver.find_element(By.XPATH,"/html/body/div[2]/main/div[2]/div/section[1]/section/nav/span[3]").click()
        sleep(1)

jogos_df

In [20]:
opts = ChromeOptions()
#opts.add_argument("--headless")
#opts.add_argument('--no-sandbox')
#opts.add_argument('--disable-dev-shm-usage')
servico=Service(ChromeDriverManager().install())
driver=webdriver.Chrome(service=servico, options=opts)

driver.get("https://ge.globo.com/futebol/brasileirao-serie-a/")
driver.maximize_window()
sleep(1)

driver.execute_script("window.scrollTo(0, 500);")
sleep(2)  # Wait for the page to load